In [1]:
import os
import numpy as np
from ili.dataloaders import StaticNumpyLoader, SummarizerDatasetLoader
from ili.inference.runner_sbi import SBIRunner
from ili.validation.runner import ValidationRunner
import netCDF4 as nc

import argparse
import torch

In [2]:
# Setting pyTorch device as GPU if available, else CPU
deviceStr = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(deviceStr)
print('Using device:', device)

Using device: cpu


In [7]:
#%% Data, parameters and cie
dataDir="/home/chartier/Documents/LTU_ILI/iliData/quijote_z0p5/Concatenated/"

# axis along which RSD has been computed (careful about confusions)
axRSD = 0

# Data Files
filePk = "PK_z0p5_RSD%i_kmax1p0.nc"%axRSD
fileBkIso = "LH_Bk_RSD%i_iso.h5"%axRSD # iso for isosceles triangles only
fileBkEqui = "LH_Bk_RSD%i_equi.h5"%axRSD # equi for equilqteral triangles only

# Explore .nc format as training
pkData = nc.Dataset(dataDir + filePk)

# Individual dimensions are accessed via dat1.dimensions['x']
for group in pkData.groups.values():
    print(group)

<class 'netCDF4._netCDF4.Group'>
group /LH_z0p5:
    dimensions(sizes): Seed(2000), multipole(3), k3D(158)
    variables(dimensions): float64 k3D(k3D), int64 multipole(multipole), int64 Seed(Seed), float64 PkRSD_ax0(Seed, multipole, k3D)
    groups: 
<class 'netCDF4._netCDF4.Group'>
group /fiducial_z0p5:
    dimensions(sizes): Seed(500), multipole(3), k3D(158)
    variables(dimensions): float64 k3D(k3D), int64 multipole(multipole), int64 Seed(Seed), float64 PkRSD_ax0(Seed, multipole, k3D)
    groups: 


In [12]:
# Just playing for netCDF4 related examples
pkLH = pkData["LH_z0p5"]
pkFid = pkData["fiducial_z0p5"]

# if we just just use Dataset.dimensions, just the names are shown without details
for dim in pkLH.dimensions.values():
  print(dim)

k3D = pkLH["k3D"]
ells = pkLH["multipole"]
seedsLH = pkLH["Seed"]
seedsFid = pkFid["Seed"]

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'Seed', size = 2000
<class 'netCDF4._netCDF4.Dimension'>: name = 'multipole', size = 3
<class 'netCDF4._netCDF4.Dimension'>: name = 'k3D', size = 158
(2000, 3, 158)


In [ ]:
# Cosmological parameters, fiducial values for prior
paramStr=[r"$\Omega_m$", r"$\Omega_b$",r"$h$",r"$n_s$",r"$\sigma_8$"]

# Means are Quijote fiducial parameters, and for the standard deviations I checked some literature
Om0 = 0.3175 ; s_Om0 = 0.0074
Ob0 = 0.049 ; s_Ob0 = 0.0003
h = 0.6711 ; s_h = 0.0054
n_s = 0.9424 ; s_ns = 0.0042
sigma_8 = 0.834 ; s_sigma_8 = 0.006
#w = -1.0 ; s_w = 0.05

# Set fiducial parameters array
fiducial = np.array([Om0, Ob0, h, n_s, sigma_8])

# For SNPE, we must distinguish the proposal prior with which the simulations were run to the prior in the Bayesian Inference
# Below are Quijote Latin Hypercubebounds
lowProp = np.array([0.1, 0.03, 0.5, 0.8, 0.6])
upProp = np.array([0.5, 0.07, 0.9, 1.2, 1.0])

In [ ]:
# reload all simulator examples as a dataloader
#all_loader = StaticNumpyLoader.from_config("configs/data/sample.yaml")
#all_loader = SummarizerDataLoader.from_config("configs/data/sample.yaml")

# train a model to infer x -> theta. save it as toy/posterior.pkl
runner = SBIRunner.from_config("configs/infer/sample_sbi.yaml")
runner(loader=all_loader)